In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm

In [2]:
import rotation_matrix as rm

In [3]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [4]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [5]:
import numpy as np

data = np.loadtxt('../pouring_simulation/output/Assembly7_TCP.txt', delimiter=',', skiprows=1)

In [6]:
data.shape[0]

981

In [7]:
#start_pos = [-0.32840330749723673,
# 0.6906975268273259,
# 0.25610656751907307,
# -1.2222948307378443,
# -1.1666822479428505,
# -1.218172452957232]

#rtde_c.moveL(start_pos, 0.5, 0.5)

In [8]:
data[500]

# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254

In [9]:
data

array([[ 1.09004862e-07,  1.91582294e-04,  1.15409000e-03],
       [ 4.48132200e-07,  3.83164080e-04,  2.30819000e-03],
       [ 1.00526850e-06,  5.74746120e-04,  3.46228000e-03],
       ...,
       [ 4.48132200e-07,  3.83164080e-04,  2.30816000e-03],
       [ 1.09004862e-07,  1.91582294e-04,  1.15407000e-03],
       [ 0.00000000e+00,  0.00000000e+00, -2.65427000e-08]])

In [10]:
start_pos = rtde_r.getActualTCPPose()

start_pos

[-0.34957822507297887,
 0.6821259171088855,
 0.22494216457758412,
 -1.2155551165943093,
 -1.176864822380718,
 -1.2292597140201593]

In [11]:
# create list of positions
positions = []
for i in range(data.shape[0]):
    positions.append([data[i,1], -data[i,0], 0.0, 0.0, 0.0, -data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

In [12]:
positions_converted = []
for i in range(data.shape[0]):
    positions_converted.append(rm.PoseTrans(start_pos, positions[i])) # transform from base to tool coordinates(positions[i]))

In [13]:
positions_converted

[[-0.34957670361022575,
  0.6821220526396402,
  0.22513370188017792,
  -1.215031198683103,
  -1.1777166727032875,
  -1.230116036067048],
 [-0.3495754122031676,
  0.6821181935146927,
  0.22532524074087398,
  -1.2145068343377101,
  -1.17856845005942,
  -1.2309722730560562],
 [-0.34957433873554855,
  0.6821143394378814,
  0.2255167818127375,
  -1.2139820325415505,
  -1.1794201395936763,
  -1.2318284100501034],
 [-0.3495734712963372,
  0.6821104905950172,
  0.22570830187858085,
  -1.213456784096329,
  -1.1802717559709284,
  -1.2326844617867798],
 [-0.3495728337180136,
  0.6821066466248731,
  0.22589984686495584,
  -1.2129310980006083,
  -1.1811232843383699,
  -1.23354041333206],
 [-0.34957241408560735,
  0.6821028077080921,
  0.22609139380859797,
  -1.2124049650405824,
  -1.1819747393579263,
  -1.23439627942016],
 [-0.34957222448070424,
  0.6820989741248017,
  0.22628294281800615,
  -1.2118783942299565,
  -1.1828261061794332,
  -1.2352520451201083],
 [-0.349572253036579,
  0.68209514605676

In [14]:
velocity = 0.5
acceleration = 0.5
blend_1 = 0.0
blend_i = 0.001
blend_3 = 0.0
path_pose1 = [start_pos[0], start_pos[1], start_pos[2], start_pos[3], start_pos[4], start_pos[5], velocity, acceleration, blend_1]
path = []
path.append(path_pose1)
for i in range(data.shape[0]-1):
    path.append([positions_converted[i][0], positions_converted[i][1], positions_converted[i][2], positions_converted[i][3], positions_converted[i][4], positions_converted[i][5], velocity, acceleration, blend_i])

path_pose3 = [positions_converted[-1][0], positions_converted[-1][1], positions_converted[-1][2], positions_converted[-1][3], positions_converted[-1][4], positions_converted[-1][5], velocity, acceleration, blend_3]
path.append(path_pose3)

In [15]:
# Send a linear path with blending in between - (currently uses separate script)
rtde_c.moveL(path)
rtde_c.stopScript()